<a href="https://colab.research.google.com/github/BAmbica39/CustomerChurnAmericanExpress/blob/main/American_Express_Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [183]:
 import numpy as np
 import pandas as pd
 import tensorflow as tf

In [184]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import joblib

# Load data
dataset = pd.read_excel("/content/American Express Dataset.xlsx")
X = dataset.iloc[:, :-1].copy()  # 10 features
y = dataset.iloc[:, -1].values
print("✅ Step 1: Loaded raw X:", X.shape)

# Label encode gender FIRST
label_encoder = LabelEncoder()
X['Gender'] = label_encoder.fit_transform(X['Gender'])
print("✅ Step 2: Encoded gender:", X.shape)
joblib.dump(label_encoder, "label_encoder.pkl")

# One-hot encode Geography (index 1 in original array, but here we use column name)
ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(), ['Geography'])],
    remainder='passthrough'
)
X_transformed = ct.fit_transform(X)
print("✅ Step 3: Column transformed shape:", X_transformed.shape)
joblib.dump(ct, "column_transformer.pkl")

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_transformed)
print("✅ Step 4: Scaled shape:", X_scaled.shape)
joblib.dump(scaler, "scaler.pkl")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)


✅ Step 1: Loaded raw X: (9927, 9)
✅ Step 2: Encoded gender: (9927, 9)
✅ Step 3: Column transformed shape: (9927, 11)
✅ Step 4: Scaled shape: (9927, 11)


In [185]:
print(X_train.shape)
print(y_train.shape)

(7941, 11)
(7941,)


ANN

In [186]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(5, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32)

model.save("churn_model.keras")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


249/249 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7779 - loss: 0.6032
Epoch 2/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8016 - loss: 0.4844
Epoch 3/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7916 - loss: 0.4638
Epoch 4/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8020 - loss: 0.4351
Epoch 5/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8105 - loss: 0.4223
Epoch 6/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8045 - loss: 0.4281
Epoch 7/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8121 - loss: 0.4160
Epoch 8/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8212 - loss: 0.4069
Epoch 9/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8243 - loss: 0.3940
Epoch 10/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8248 - loss: 0.4020
Epoch 11/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8289 - loss: 0.3935
Epoch 12/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

predictions

single prediction

In [178]:
#print(ann.predict(sc.transform([[0.0, 1.0, 0.0, 501, 0, 32, 2, 0.0, 4, 1, 545501]])) >0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
[[False]]


In [187]:
y_pred= ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[0 1]
 [0 0]
 [0 0]
 ...
 [0 1]
 [0 0]
 [1 1]]


In [188]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1506   67]
 [ 213  200]]


0.8590130916414904

In [194]:
from tensorflow.keras.models import load_model
import joblib

model = load_model("churn_model.keras")
label_encoder = joblib.load("label_encoder.pkl")
column_transformer = joblib.load("column_transformer.pkl")
scaler = joblib.load("scaler.pkl")

def predict_churn(credit_score, geography, gender, age, customer_since,
                  current_account, num_products, upi_enabled, estimated_income):
    try:
        input_row = {
            'Credit Score': float(credit_score),
            'Geography': geography,
            'Gender': gender,
            'Age': int(age),
            'Customer Since': int(customer_since),
            'Current Account': float(current_account),
            'Num of products': int(num_products),
            'UPI Enabled': int(upi_enabled),
            'Estimated Yearly Income': float(estimated_income)
        }

        df_input = pd.DataFrame([input_row])

        # Label encode gender
        df_input['Gender'] = label_encoder.transform(df_input['Gender'])

        # OneHot transform geography + passthrough
        transformed = column_transformer.transform(df_input)

        # Scale features
        scaled = scaler.transform(transformed)

        # Make prediction
        prediction = model.predict(scaled)[0][0]
        return "Will Churn" if prediction > 0.5 else "Will Not Churn"

    except Exception as e:
        return f"❌ Error: {str(e)}"



In [195]:
print(dataset.columns.tolist())


['Credit Score', 'Geography', 'Gender', 'Age', 'Customer Since', 'Current Account', 'Num of products', 'UPI Enabled', 'Estimated Yearly Income', 'Closed']


In [196]:
# Will likely NOT churn
new_customer_1 = [700, "Delhi", "Male", 40, 6, 150000, 2, 1, 70000]
print(predict_churn(*new_customer_1))

# Will likely CHURN
new_customer_2 = [450, "Mumbai", "Female", 60, 2, 0, 1, 0, 30000]
print(predict_churn(*new_customer_2))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Will Not Churn
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Will Churn


In [197]:
pip install gradio

In [198]:
import gradio as gr
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# Load model and preprocessing objects
model = load_model("churn_model.keras")
label_encoder = joblib.load("label_encoder.pkl")
column_transformer = joblib.load("column_transformer.pkl")
scaler = joblib.load("scaler.pkl")

def predict_churn(credit_score, geography, gender, age, customer_since,
                  current_account, num_products, upi_enabled, estimated_income):
    try:
        input_row = {
            'Credit Score': float(credit_score),
            'Geography': geography,
            'Gender': gender,
            'Age': int(age),
            'Customer Since': int(customer_since),
            'Current Account': float(current_account),
            'Num of products': int(num_products),
            'UPI Enabled': int(upi_enabled),
            'Estimated Yearly Income': float(estimated_income)
        }

        df_input = pd.DataFrame([input_row])

        # Encode Gender
        df_input['Gender'] = label_encoder.transform(df_input['Gender'])

        # Transform Geography and scale
        transformed = column_transformer.transform(df_input)
        scaled = scaler.transform(transformed)

        prediction = model.predict(scaled)[0][0]
        return "💔 Will Churn" if prediction > 0.5 else "✅ Will Not Churn"

    except Exception as e:
        return f"❌ Error: {str(e)}"

# Gradio interface
demo = gr.Interface(
    fn=predict_churn,
    inputs=[
        gr.Number(label="Credit Score"),
        gr.Dropdown(["Delhi", "Mumbai", "Bengaluru"], label="Geography"),
        gr.Dropdown(["Male", "Female"], label="Gender"),
        gr.Number(label="Age"),
        gr.Number(label="Customer Since"),
        gr.Number(label="Current Account Balance"),
        gr.Number(label="Num of Products"),
        gr.Dropdown([0, 1], label="UPI Enabled (1 = Yes, 0 = No)"),
        gr.Number(label="Estimated Yearly Income")
    ],
    outputs=gr.Textbox(label="Churn Prediction"),
    title="📊 Customer Churn Predictor",
    description="Enter customer details to predict if they will churn."
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://426e528c849e66b206.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
